## Merging idExpedicion from perfiles_df to etapas_df

1. Loading and processing etapas_df
2. Loading and processing perfiles_df
3. Simplifying etapas_df
4. Simplifying perfiles_df
5. <font color='red'>MERGE</font>
6. Print full_info etapas_df

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import pandas as pd
import time
import DailyEtapasBuilder
import DailyPerfilesBuilder
from Utils import TransantiagoConstants

In [4]:
dates = TransantiagoConstants.updateCurrentSSHDates()

In [5]:
summary_dir = TransantiagoConstants.SummaryDir

selected_services_path = os.path.join(summary_dir,'0_servicios_seleccionados.xlsx')
selected_services_df = pd.read_excel(selected_services_path)

selected_services = selected_services_df.loc[:,'FINAL_CODES']
selected_services = [str(x) for x in selected_services]

In [15]:
start_index = dates.index('2017-07-11')

In [16]:
end_index = dates.index('2017-08-27')

In [17]:
dates = dates[start_index:end_index+1]

In [18]:
dates

['2017-07-11',
 '2017-07-12',
 '2017-07-13',
 '2017-07-14',
 '2017-07-15',
 '2017-07-16',
 '2017-07-18',
 '2017-07-19',
 '2017-08-01',
 '2017-08-02',
 '2017-08-03',
 '2017-08-04',
 '2017-08-05',
 '2017-08-06',
 '2017-08-08',
 '2017-08-09',
 '2017-08-11',
 '2017-08-12',
 '2017-08-13',
 '2017-08-14',
 '2017-08-15',
 '2017-08-17',
 '2017-08-18',
 '2017-08-19',
 '2017-08-20',
 '2017-08-21',
 '2017-08-22',
 '2017-08-23',
 '2017-08-24',
 '2017-08-25',
 '2017-08-26',
 '2017-08-27']

In [19]:
for date in dates:
    print(date + ' is being processed')
    if ((date!='2017-03-14')&(date!='2017-04-10')&(date!='2017-05-25')&(date!='2017-07-11')):
        ### BUILDING AND PROCESSING ETAPAS ###
        etapas_builder = DailyEtapasBuilder.RunSilentlyDailyEtapasBuilderClass(date)
        etapas_builder.loadSimplifiedEtapas()
        etapas_builder.mergeTurnstileData()
        etapas_builder.cleanDataFrame() #Be aware of this, as it deletes all rows with missing values.
        etapas_builder.sortDataFrame()
        etapas_builder.postProcessingSortedDataFrame()
        processed_sorted_df = etapas_builder.etapas_df
        simplified_etapas = processed_sorted_df.loc[processed_sorted_df['servicio_subida'].str.contains('|'.join(selected_services)),:]
        ### BUILDING AND PROCESSING PERFILES ###
        perfiles_builder = DailyPerfilesBuilder.RunSilentlyDailyPerfilesBuilderClass(date)
        perfiles_builder.runSimplifyPerfiles()
        perfiles_builder.loadSimplifiedPerfiles()
        perfiles_builder.dropParaderos()
        perfiles_df = perfiles_builder.perfiles_df
        simplified_perfiles = perfiles_df.loc[perfiles_df['ServicioSentido'].str.contains('|'.join(selected_services)),:]
        ### CRITICAL ###
        for index, row in simplified_perfiles.iterrows():
            servicio = row['ServicioSentido']
            patente = row['Patente']
            Hini = row['Hini']
            Hfin = row['Hfin']
            idExpedicion = row['idExpedicion']
            simplified_etapas.loc[(simplified_etapas['servicio_subida']==servicio)&(simplified_etapas['sitio_subida']==patente)&(Hini<=simplified_etapas['t_subida'])&(simplified_etapas['t_subida']<=Hfin),'idExpedicion'] = idExpedicion
        
        ### PRINTING TO A FILE ###
        full_etapas_name = date + '_full_etapas.csv'
        full_etapas_path = os.path.join(summary_dir, full_etapas_name)
        simplified_etapas.to_csv(full_etapas_path, sep=';', encoding='latin-1')
    else:
        pass

2017-07-11 is being processed
2017-07-12 is being processed
Not found in BOTH turnstile databases: 1335554


C:\Users\Tesista\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Tesista\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2017-07-13 is being processed
Not found in BOTH turnstile databases: 1243924
2017-07-14 is being processed
Not found in BOTH turnstile databases: 1321812
2017-07-15 is being processed
Not found in BOTH turnstile databases: 626508
2017-07-16 is being processed
Not found in BOTH turnstile databases: 467141
2017-07-18 is being processed
Not found in BOTH turnstile databases: 1268969
2017-07-19 is being processed
Not found in BOTH turnstile databases: 1299965
2017-08-01 is being processed
Not found in BOTH turnstile databases: 1401105
2017-08-02 is being processed
Not found in BOTH turnstile databases: 1435747
2017-08-03 is being processed
Not found in BOTH turnstile databases: 1450241
2017-08-04 is being processed
Not found in BOTH turnstile databases: 1472091
2017-08-05 is being processed
Not found in BOTH turnstile databases: 928636
2017-08-06 is being processed
Not found in BOTH turnstile databases: 557883
2017-08-08 is being processed
Not found in BOTH turnstile databases: 1485697
201

## Closed